In [ ]:
## Italy
- https://www.kaggle.com/sudalairajkumar/covid19-in-italy

In [ ]:
df_italy = pd.read_csv(local_dir + 'covid19-in-italy/covid19_italy_region.csv')

df_italy['Date'] = pd.to_datetime(df_italy['Date'])
df_italy['year'] = df_italy['Date'].dt.year
df_italy['month'] = df_italy['Date'].dt.month
df_italy['day'] = df_italy['Date'].dt.day

df_italy.drop(['SNo','Date','Country'], axis=1, inplace=True)
df_italy = df_italy.groupby(['year','month','day']).sum().reset_index()
df_italy.head()